In [1]:
##edits of jason's getConcentration from laminarflow class, trying equation from today's talk

def getConcentration(self):
        """
        A method designed to determine the bluk concentration in the intestines as it passes through the intestines.

        Outputs:
        ------------------------------------------------------------------------------------
        lengths: a list of length n that has the legnths at which the Graetz numbers were calculated at
        serGraetz: a list of Graetz numbers for ser (length n)
        trypGraetz: a list of Graetz numbers for tryp (length n)
        """
        self.serMBetaList = serMBetaList = interpolateForValue(self.serEffPerm, self.MBeta)
        self.trypMBetaList = trypMBetaList = interpolateForValue(self.trypEffPerm, self.MBeta)
        for i in range(len(self.serConcentration)-1):

            # The initialize delta ser and tryp
            serDelta = 0
            trypDelta = 0

            # Calculation of delta ser and tryp over the first five terms of the series.
            for j in range(5):
                serDelta += serMBetaList[j+5]*np.exp(-1*serMBetaList[j]**2*self.serGraetz[i]) #Beta * 5 first in file, then M * 5
                trypDelta += trypMBetaList[j+5]*np.exp(-1*trypMBetaList[j]**2*self.trypGraetz[i])

            #REACTTTTTT
            c = (self.serConcentration[i], self.HTPConcentration[i], self.trypConcentration[i])
            serRate, HTPRate, trypRate = multistep(c, *self.kinetics)

            # Adding the concentration calculation to the list
            self.HTPConcentration[i+1] = self.HTPConcentration[i] + HTPRate*self.dt

            self.serConcentration[i+1] = self.serConcentration[i] * serDelta
            self.serConcentration[i+1] += serRate*self.dt

            self.trypConcentration[i+1] = self.trypConcentration[i] * trypDelta
            self.trypConcentration[i+1] += trypRate*self.dt
            

            
            

In [12]:
#initialize 3D vector space over cylindrical coordinates
import numpy as np


def deltaZ(Z):
    '''function to calculate second derivative by z'''
    Zleft = Z[1:-1,0:-2]
    Zright = Z[1:-1,2:]
    Zcenter = Z[1:-1,1:-1]
    return (Zleft - 2*Zcenter + Zright)

# def deltaR(Z):
#     '''Functions to calculate second derivative by r'''
#     Ztop = Z[0:-2,1:-1]
#     Zbottom = Z[2:,1:-1]
#     Zcenter = Z[1:-1,1:-1]
# I'm not totally sure how to do this with R at simultaneously
    
    
    
def crossSecAreas(self,rings, velocity_max):
    "gives an array of radial ring areas for a tube based on given radius and number of subsections"
    totarea=np.zeros(rings)
    velocityProfile=np.zeros(rings)
    r=self.radius/rings
    for m in range(rings):
        totarea[m]=np.pi*(r*(m+1))**2 - np.pi*(r*(m))**2
        velocityProfile[m] = velocity_max*(self.radius**2-(r*m)**2)    
    return totarea

def getConcentration(self):

    trypconc = np.zeros((len(time), rings, sections))
    serotonconc = np.zeros_like(trypconc)
    htpconc = np.zeros_like(trypconc)

    
    # rings - dr
    # sections - dz
    trypconc[0,:,0] = trypConditions.concentration
    serotonconc[0,:,0] = serConditions.concentration
    
    totalSerotoninUptake = 0
    
    for i in range(len(time)):
        #convection
        
        for j in range(rings):
            for k in range(sections-1):
                
                #velocity based convection, each subsection per time step
                #flowrate volumetric, dim=rings m3/s
                volflows=totarea*velocityProfile
                #section volume
                vols=totarea*(length/sections)
                
                trypconc[j,k+1]= trypconc[j,k]-(trypconc[j,k]/vols[j]*volflows[j])
                serotonconc[j,k+1] = serotonconc[j,k]-(serotonconc[j,k]/vols[j]*volflows[j])
    
        #microbial reaction over whole cross section
        #tryptopphan converstion in each subsection per time step
        # is there a .* function for matrices like matlab??
        #get rates for each section....
        
        for j in range(rings):
            for k in range(sections-1):
                trypconc[j,k]=trypcond[j,k]-rate
                serotonconc[j,k]=serotonconc[j,k]+rate

        #reaction at the wall
        #just for outer rings at all lengths, how do you factor in surface area??
        for m in range(sections):
            trypconc(rings,m)=trypconc(rings,m)-trypconc(rings,m)*rxnrateWall
            serotonconc(rings,m)=+trypconc(rings,m)*rxnrateWall
    
        #diffusion into wall - keep track of
        #just for outer rings, combine with above?
        #again, how to factor in wall surface area
        
        for n in range(sections):
            trypconc(rings,n)=trypconc(rings,n)-trypconc(rings,n)*tryppermeability
            serotonconc(rings,n)=- serotonconc(rings,n) - serotonconc(rings,n)*seropermeability         
            totalSerotoninUptake =+ serotonconc(rings,n*seropermeability)

